## Add magic command to find exectution time

In [1]:
%load_ext autotime

## Imports

In [71]:
from math import log
from collections import defaultdict
from collections import Counter
from copy import deepcopy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

time: 1.17 ms


## Define Shannon's entropy

In [56]:
dataset = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
labels = ['no_surfacing','flippers']

time: 1.45 ms


In [4]:
label_counts = defaultdict(int)
for data_point in dataset:
    label = data_point[-1]
    label_counts[label] += 1
label_counts

defaultdict(int, {'yes': 2, 'no': 3})

time: 363 ms


entropy = - (sum of all classes (p * log2(p)))

In [5]:
len_data_points = len(dataset)
shannon_entropy = 0
for label, occ in label_counts.items():
    prob = occ / len_data_points
    shannon_entropy -= prob * log(prob, 2)
shannon_entropy

0.9709505944546686

time: 278 ms


Function is written considering last item would be label

In [6]:
def entropy(dataset):
    label_counts, entropy = defaultdict(int), 0
    for data_point in dataset:
        label_counts[data_point[-1]] += 1
    for label, occ in label_counts.items():
        prob = occ / len(dataset)
        entropy -= prob * log(prob, 2)
    return entropy

time: 325 ms


The higher the entropy, the more mixed up the data is. Let’s make the data a little
messier and see how the entropy changes. We’ll add a third class, which is called
maybe , and see how the entropy changes

In [7]:
dataset = [[1, 1, 'maybe'], [1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
entropy(dataset)
print("We observe that by adding a new class, that is by making data more messier, entropy increases")

1.4591479170272448

We observe that by adding a new class, that is by making data more messier, entropy increases
time: 768 ms


## Lets split the data on any feature column and value!

On the simple dataset that we have, lets try to split on the first column("no_surfacing"). It would be split based on if the value is either 1 or 0

In [8]:
dataset = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

time: 207 ms


Lets try to split data on first column if value is 1

In [9]:
axis = 0
split_dataset = list()
for data_point in dataset:
    if data_point[axis] == 1:
        new_split = data_point[:axis] + data_point[axis + 1:]
        split_dataset.append(new_split)
split_dataset

[[1, 'yes'], [1, 'yes'], [0, 'no']]

time: 278 ms


The first three elements come to this group!

In [10]:
def split_dataset(dataset, axis, value):
    split_dataset = [data_point[:axis] + data_point[axis + 1:] 
                     for data_point in dataset if data_point[axis] == value]
    return split_dataset

time: 959 µs


In [11]:
split_dataset(dataset, 0, 0)

[[1, 'no'], [1, 'no']]

time: 3.14 ms


## Lets find on which fetaure the data has to be split!

In [12]:
num_features = len(dataset[0]) - 1
best_infogain = float("-Inf")
split_feature_index = -1
base_entropy = entropy(dataset)
print(f"Base entropy : {base_entropy}")

Base entropy : 0.9709505944546686
time: 1.95 ms


In [13]:
for axis in range(num_features):
    unique_feature_val = {data_point[axis] for data_point in dataset}
    feature_entropy = 0
    for val in unique_feature_val:
        sub_dataset = split_dataset(dataset, axis, val)
        prob = len(sub_dataset) / len(dataset)
        feature_entropy += prob * entropy(sub_dataset)
    info_gain = base_entropy - feature_entropy
    if info_gain > best_infogain:
        best_infogain = info_gain
        split_feature_index = axis
print(f"Infogain : {best_infogain}")
print(f"Feature index where split gives maximum information gain : {split_feature_index}")

Infogain : 0.4199730940219749
Feature index where split gives maximum information gain : 0
time: 3.19 ms


In [14]:
def best_feature(dataset):
    num_features = len(dataset[0]) - 1
    best_infogain = float("-Inf")
    split_feature_index = -1
    base_entropy = entropy(dataset)
    
    for axis in range(num_features):
        unique_feature_val = {data_point[axis] for data_point in dataset}
        feature_entropy = 0
        for val in unique_feature_val:
            sub_dataset = split_dataset(dataset, axis, val)
            prob = len(sub_dataset) / len(dataset)
            feature_entropy += prob * entropy(sub_dataset)
        info_gain = base_entropy - feature_entropy
        if info_gain > best_infogain:
            best_infogain = info_gain
            split_feature_index = axis
            
    return split_feature_index

time: 2.39 ms


In [15]:
best_feature(dataset)

0

time: 3.53 ms


## Lets build the tree

In [28]:
unique_labels = {data_point[-1] for data_point in dataset}
unique_labels

{'no', 'yes'}

time: 2.91 ms


Base condition to stop recursion when all group contains only one class, This is because if only one class is present, the entropy would be 0 hence, information gain would be maximum

In [29]:
if len(unique_labels) == 1: pass
    #return the class label

time: 807 µs


When only label is present, meaning the dataset is enough split, return most common label as the class label

In [20]:
if len(dataset[0]) == 1:
    label = Counter([data_point[-1] for data_point in dataset]).most_common()[0][0]

time: 1.29 ms


In [31]:
split_feature = best_feature(dataset)
print(f"The dataset has to be split on the feature : {labels[split_feature]}")

The dataset has to be split on the feature : no_surfacing
time: 869 µs


In [55]:
def create_tree(dataset, labels):
    unique_labels = {data_point[-1] for data_point in dataset}
    if len(unique_labels) == 1:
        return Counter(unique_labels).most_common()[0][0]
    if len(dataset[0]) == 1:
        return Counter([data_point[-1] for data_point in dataset]).most_common()[0][0]
        
    split_feature_ind = best_feature(dataset)
    split_feature = labels[split_feature_ind]
    del labels[split_feature_ind]
    
    tree = defaultdict(dict)
    unique_vals = {data_point[split_feature_ind] for data_point in dataset}
    for val in unique_vals:
        sub_labels = labels[:]
        sub_dataset = split_dataset(dataset, split_feature_ind, val)
        tree[split_feature][val] = create_tree(sub_dataset, sub_labels)
    return tree

time: 2.67 ms


In [73]:
dataset = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
labels = ['no_surfacing','flippers']

time: 856 µs


In [74]:
tree = create_tree(dataset, deepcopy(labels))
tree

defaultdict(dict,
            {'no_surfacing': {0: 'no',
              1: defaultdict(dict, {'flippers': {0: 'no', 1: 'yes'}})}})

time: 4.32 ms


## Classify a new point using this tree

In [85]:
def classify(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__=='dict':
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel

time: 3.03 ms
